In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os import path
from pathlib import Path  

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [2]:
# import files
data_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/down_data_matrix.csv') # Data matrix
gene_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Homo_sapiens.tsv') # Gene Symbol and ID
pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Pathway_Down.csv') # all the pathways used in the heatmap
gene_to_pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Gene_to_Pathway.csv') # every pathway and the relevant genes

# dataframe of metadata
data = pd.read_csv(data_file,index_col=0)
gene_info = pd.read_csv(gene_file, sep="\t")
pathway = pd.read_csv(pathway_file,index_col=0).fillna("0")
gene_to_pathway = pd.read_csv(gene_to_pathway_file)

In [3]:
pathways = pathway.index.tolist()
pathway_name = []
pathway_number = []
for i in range(len(pathways)):
    temp = pathway.iloc[i]
    temp2 = []
    for j in range(4):
        if temp[j] != "0":
            pathway_name.append(pathways[i])
            pathway_number.append(temp[j])
pathway_number = set(pathway_number)            

In [4]:
# get gene IDs
# note that 7 percent (~252/3418 genes does not have matching symbol=ID)
Gene_IDs = []
for i in range(len(data.index.tolist())):
    Gene_name = data.iloc[i][0]
    
    # get gene symbol
    if "," in Gene_name:
        temp = Gene_name.split(",")
        if temp[0] != "":
            Gene_name = temp[0]
        else:
            Gene_name = temp[1].lstrip()
    
    # gene_symbol to ID
    ID = gene_info.loc[gene_info['Symbol'] == Gene_name]['GeneID'].tolist()
    if ID == []:
        Gene_IDs.append("None")
    else:
        Gene_IDs.append(ID[0])

In [5]:
# add the IDs column in and drop the genes that doesn't have an ID
data["ID"] = Gene_IDs
data = data[data['ID'] != "None"]

In [6]:
# get the pathway
# only 1402/3166 genes belong to a functional pathway
pathways_by_gene = []
pathways_by_gene_column = []
for i in range(len(data.index.tolist())):
    temp_ID = data.iloc[i][28]
    
    # get corresponding pathways
    temp_pathways = gene_to_pathway[gene_to_pathway['Gene']== data.iloc[i][28]]["Pathway"].tolist()
    
    if len(temp_pathways) == 0:
        pathways_by_gene_column.append("none")
    else:
        pathways_by_gene.append(temp_pathways)
        pathways_by_gene_column.append(" ".join(temp_pathways))

In [7]:
data["All Pathways"] = pathways_by_gene_column
data = data[data['All Pathways'] != "none"]

## output the data file as long as the gene is involved with a pathways ##

In [8]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Data_All_Pathways.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)  

## continue to shrink data only if a gene is involved with a pathway actually used in the heatmap ##

In [55]:
# find the pathway that's involved with the heat map
pathways_involved = []
pathways_involved_column = []
for i in range(len(data.index.tolist())):
    a = pathways_by_gene[i]
    temp = []

    for j in a:
        if j in pathway_number:
            temp.append(j)
    
    if len(temp) == 0:
        pathways_involved_column.append("none")
    else:
        pathways_involved.append(temp)
        pathways_involved_column.append(" ".join(temp))

In [56]:
data["Involved Pathways"] = pathways_involved_column
data = data[data['Involved Pathways'] != "none"]

In [57]:
drugs = data.columns.values.tolist()[1:28]
functions = pathway.index.tolist()

In [58]:
# create the finalmatrix
final_matrix = [[0.0] * len(drugs)]*len(functions)
counter=0
for i in range(len(data.index.tolist())):
    temp_pathways = pathways_involved[i]
    temp_values = data.iloc[i].tolist()[1:28]
    temp_functions = []
    for j in temp_pathways:
        a = pathway[pathway["pathway_1"] == j].index.tolist()
        if len(a)>0:
            temp_functions.append(a[0])
        else:
            a = pathway[pathway["pathway_2"] == j].index.tolist()
            if len(a)>0:
                temp_functions.append(a[0])
            else:
                a = pathway[pathway["pathway_3"] == j].index.tolist()
                if len(a)>0:
                    temp_functions.append(a[0])
                else:
                    a = pathway[pathway["pathway_4"] == j].index.tolist()
                    if len(a)>0:
                        temp_functions.append(a[0])
    for k in temp_functions:
        counter = counter + 1
        temp_index = functions.index(k)
        final_matrix[temp_index] = [final_matrix[temp_index][n] + temp_values[n] for n in range(len(temp_values))] 


In [59]:
final_data_functions_by_drugs = pd.DataFrame(final_matrix,index = functions, columns=drugs)
final_data_functions_by_drugs

Trastuzumab  Tideglusib  Lapatinib  Mirdametinib  \
Apelin                  0.000000    0.000000   6.572377      0.000000   
Insulin                -9.216824    0.000000  39.282873     17.632998   
PI3K-AKT               -4.630123    0.000000  47.539405      9.004597   
mTOR                    0.000000    0.000000  49.070156      0.000000   
HIF-1                   0.000000    0.000000  20.172436      0.000000   
AMPK                   -9.216824    0.000000  33.627426      8.628401   
Hippo                   0.000000    0.000000  26.354122      0.000000   
Rig-I like receptor     0.000000    0.000000   7.163843      0.000000   
NF-kappa B              0.000000    0.000000   4.756414      0.000000   
Calcium                 0.000000    0.000000  19.408772      0.000000   
NOD-like receptor       0.000000    0.000000   6.812462      0.000000   
Toll-llike receptor     0.000000    0.000000   0.000000      0.000000   
TNF                     0.000000    0.000000  13.538881      9.004597   
Rap1                    0.000000    0.000000  34.758592      9.145954   
IL-17                   0.000000    0.000000   0.000000      0.000000   
Estrogen                0.000000    0.000000  19.701231      9.004597   
Sphingolipid            0.000000    0.000000   0.000000      9.613984   
Oxytocin                0.000000    0.000000  24.877067      0.000000   
VEGF                    0.000000    0.000000   0.000000      0.000000   
cGMP                    0.000000    0.000000  13.298796      9.004597   
TCR                     0.000000    0.000000  11.396723      0.000000   
BCR                     0.000000    0.000000   4.756414      0.000000   
Chemokine               0.000000    0.000000  25.294483      0.000000   
Neurotrophin            0.000000    0.000000  18.722106      8.679737   
Prolactin               0.000000    0.000000   6.688382      0.000000   
Phospholipase D         0.000000    0.000000  12.974812     18.293720   
Ras                     0.000000    0.000000   6.402435     18.293720   
Fc epsilon RI           0.000000    0.000000   0.000000      9.613984   
GnRH                    0.000000    0.000000  12.974812      0.000000   
MAPK                    0.000000    0.000000  58.248391     18.232321   
ErbB                    0.000000    0.000000  32.206160      8.679737   
FoxO                    0.000000    0.000000  18.274003      0.000000   
cAMP                    0.000000    0.000000  11.755562      0.000000   
Jak-STAT                0.000000    0.000000  13.443873      0.000000   
Notch                   0.000000    7.564789   6.319840      0.000000   
Glucagon               -9.216824    0.000000   6.726419      9.004597   
Wnt                    -4.921164    7.564789   7.041438      0.000000   
TGF-beta                0.000000    0.000000   6.892026      9.588664   
Thyroid                 0.000000    0.000000  30.495374      9.004597   

                       MK-2206  Nintedanib  Pertuzumab  Pictilisib  \
Apelin                0.000000    0.000000    0.000000    0.000000   
Insulin              67.141630    5.538034   -9.541298   24.086317   
PI3K-AKT             32.525836    0.000000  -14.223007    6.509195   
mTOR                 24.449587    0.000000  -13.840524    7.003734   
HIF-1                 0.000000    0.000000   -4.909324    6.509195   
AMPK                 39.638972   10.847220  -13.712845   29.850270   
Hippo                14.844044    5.550695   -9.168958   13.890178   
Rig-I like receptor   8.712568    0.000000    0.000000    0.000000   
NF-kappa B           12.378042    9.355389    0.000000    6.945888   
Calcium               8.457309    0.000000    0.000000    6.509195   
NOD-like receptor    12.123614   10.727500   -4.502647    7.175220   
Toll-llike receptor   6.966162   10.172461    0.000000    6.945888   
TNF                  13.382445   10.727500   -9.134621    7.175220   
Rap1                  0.000000    0.000000    0.000000    6.509195   
IL-17                15.385197   11.254447    0.00

In [60]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Final_data_functions_by_drugs.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_data_functions_by_drugs.to_csv(filepath)  

## establish matrix as pathway by drugs ##